Sobre o Dataset

Temos em mãos um arquivo com dados de pacientes que desenvolveram ou não diabetes. Precisamos gerar uma amostra de dados com os pacientes com mais de 50 anos e para cada um deles indicar em uma nova coluna se o paciente está normal (BMI menor que 30) ou obeso (BMI maior ou igual a 30). Então devemos gerar um novo arquivo CSV.

Instalando e carregando pacotes

In [ ]:
!pip install -q -U watermark
!pip install -q ipython-sql

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
%reload_ext watermark
%watermark -a "Data Nerd" --iversions

In [ ]:
# Criar banco de dados sqlite e conectar
cnn = sqlite3.connect('database/dbprojeto.db')

In [ ]:
df = pd.read_csv('csv/diabetes.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Copia o dataframe para dentro do banco de dados como uma tabela
df.to_sql('diabetes',cnn)

In [ ]:
%load_ext sql
%sql sqlite:///database/dbprojeto.db

In [ ]:
%%sql

SELECT COUNT(*) FROM diabetes

In [ ]:
df.columns

In [ ]:
%%sql

CREATE TABLE pacientes (Pregnancies INT,
                        Glucose INT,
                        BloodPressure INT,
                        SkinThickness INT,
                        Insulin INT,
                        BMI decimal(8,2),
                        DiabetesPedigreeFunction decimal(8,2),
                        Age INT,
                        Outcome INT);

In [ ]:
%%sql

INSERT INTO pacientes(Pregnancies,
                        Glucose,
                        BloodPressure,
                        SkinThickness,
                        Insulin,
                        BMI,
                        DiabetesPedigreeFunction,
                        Age,
                        Outcome)
SELECT Pregnancies,
       Glucose,
       BloodPressure,
       SkinThickness,
       Insulin,
       BMI,
       DiabetesPedigreeFunction,
       Age,
       Outcome
FROM diabetes WHERE Age > 50;

In [ ]:
%%sql
select * from pacientes

In [ ]:
%%sql

ALTER TABLE pacientes
ADD Perfil VARCHAR(10);

In [ ]:
%%sql

UPDATE pacientes
SET Perfil = 'Normal'
WHERE BMI < 30;

In [ ]:
%%sql

UPDATE pacientes
SET Perfil = 'Obeso'
WHERE BMI >= 30;

In [ ]:
%%sql

SELECT * FROM pacientes

Exportando os dados para CSV

In [ ]:
# Query
query = cnn.execute("SELECT * FROM pacientes")
query

In [ ]:
# Criando Lista
cols = [coluna[0] for coluna in query.description]
cols

In [ ]:
# Gerandos o dataframe
resultado = pd.DataFrame.from_records(data =  query.fetchall(), columns = cols)

In [ ]:
# Shape
resultado.shape

In [ ]:
# Visualizando uma pequema amostra dos dados
resultado.head()

In [ ]:
# Salva em CSV - Etapa final
resultado.to_csv('csv/pacientes.csv', index = False)

FIM